In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold
from scipy.stats import sem
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
dir_path = '/content/drive/MyDrive/DataScience/SIGNATE/健診データによる肝疾患判定/データ/'
df_train = pd.read_csv(dir_path + 'train.csv')
df_test = pd.read_csv(dir_path + 'test.csv')

In [3]:
df_train_ = df_train.copy()
df_train_['Gender'] = df_train_['Gender'].replace({'Male':0,'Female':1})
df_train_

,Age,Gender,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,disease
0,59,0,0.786886,0.150498,220.178691,13.467617,21.729246,6.815731,3.112276,1.006802,0
1,69,0,1.003987,0.195625,221.218413,51.033462,64.752323,6.889608,3.051521,0.751346,1
2,65,0,0.657140,0.081287,320.770533,12.625011,30.613180,5.947767,2.489167,0.774952,0
3,65,0,0.906822,0.214173,369.278160,34.347597,54.510085,6.967183,3.613837,0.988155,1
4,22,1,1.734959,0.197706,222.782025,20.572891,170.010177,5.837537,3.068697,1.026654,0
...,...,...,...,...,...,...,...,...,...,...,...
845,25,0,0.767450,0.186467,215.656126,13.963269,20.160997,6.824975,3.622443,1.297731,0
846,57,0,0.867946,0.156997,213.810275,14.564305,20.122535,7.509893,3.619143,1.296843,0
847,50,0,0.826785,0.138898,230.340081,40.495102,49.620857,7.529840,3.724408,0.987033,1
848,32,1,0.761972,0.137969,214.170393,18.048321,54.447774,5.944855,3.150978,1.220787,0


In [4]:
df_test_ = df_test.copy()
df_test_['Gender'] = df_test['Gender'].replace({'Male':0,'Female':1})
df_test_

,Age,Gender,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio
0,25,0,0.801797,0.186800,214.448679,15.260516,19.496649,6.817903,3.600752,1.288006
1,26,1,0.834857,0.116426,234.136816,10.509159,16.733312,6.795433,3.111043,1.030562
2,64,0,0.791830,0.197940,216.039909,14.578304,20.695866,7.524030,3.627621,1.278579
3,65,0,0.834003,0.175741,200.901130,12.820356,20.102517,6.829109,3.626758,1.275070
4,34,1,0.690296,0.135503,208.705835,20.279018,25.096685,8.663530,4.170750,1.280078
...,...,...,...,...,...,...,...,...,...,...
345,65,1,0.913867,0.120555,214.354728,16.964820,28.723677,6.960649,3.505619,1.224655
346,59,0,0.818573,0.206075,215.898729,14.733461,20.470517,7.531071,3.637508,1.282833
347,25,0,0.787054,0.186129,214.167523,14.936585,20.589993,7.505128,3.619033,1.287256
348,64,0,0.725194,0.191307,220.272862,17.532595,48.115014,7.559414,3.635871,1.263348


In [5]:
x = df_train_.drop('disease',axis=1)
x

,Age,Gender,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio
0,59,0,0.786886,0.150498,220.178691,13.467617,21.729246,6.815731,3.112276,1.006802
1,69,0,1.003987,0.195625,221.218413,51.033462,64.752323,6.889608,3.051521,0.751346
2,65,0,0.657140,0.081287,320.770533,12.625011,30.613180,5.947767,2.489167,0.774952
3,65,0,0.906822,0.214173,369.278160,34.347597,54.510085,6.967183,3.613837,0.988155
4,22,1,1.734959,0.197706,222.782025,20.572891,170.010177,5.837537,3.068697,1.026654
...,...,...,...,...,...,...,...,...,...,...
845,25,0,0.767450,0.186467,215.656126,13.963269,20.160997,6.824975,3.622443,1.297731
846,57,0,0.867946,0.156997,213.810275,14.564305,20.122535,7.509893,3.619143,1.296843
847,50,0,0.826785,0.138898,230.340081,40.495102,49.620857,7.529840,3.724408,0.987033
848,32,1,0.761972,0.137969,214.170393,18.048321,54.447774,5.944855,3.150978,1.220787


In [6]:
y = df_train_['disease']
y

0      0
1      1
2      0
3      1
4      0
      ..
845    0
846    0
847    1
848    0
849    0
Name: disease, Length: 850, dtype: int64

In [9]:
lgc = LogisticRegression(random_state=0)
forest = RandomForestClassifier(n_estimators=100,random_state=0)
xgb = XGBClassifier(random_state=0)
gbdt = GradientBoostingClassifier(random_state=0)

estimators=[('rf', forest),
           ('xg', xgb),
            ('gb', gbdt)]

In [10]:
x_train, x_test, t_train, t_test = train_test_split(x,y,test_size=0.3,random_state=0)

In [11]:
model = StackingClassifier(estimators=estimators, final_estimator=lgc, cv=3)
model.fit(x_train, t_train)

StackingClassifier(cv=3,
                   estimators=[('rf', RandomForestClassifier(random_state=0)),
                               ('xg',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              gpu_id=None, grow_policy=None,
                                              importanc...
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=0, ...)),
                               ('gb',
                                GradientBoostingClassifier(random_state=0))],
                   final_estimator=LogisticRegression(random_state=0))

In [12]:
pre = model.predict(x_test)

# for i in range(211):
#     if pre[i] >= 0.36:
#         pre[i] = 1
#     else:
#         pre[i] = 0

pre

array([1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1])

In [13]:
print('AUC: {}'.format(roc_auc_score(t_test,pre)))

AUC: 0.8861904761904763


In [14]:
prediction = model.predict(df_test_)
prediction

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0,

In [15]:
submission = df_test.copy()
submission = submission.reset_index()
submission
df = pd.DataFrame(submission['index'])
df['disease'] = prediction
df

,index,disease
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
345,345,0
346,346,0
347,347,0
348,348,0


In [16]:
df.to_csv('submission_stac_2.csv',index=False,header=False)